In [1]:
# Imporing required libraries
from msilib.schema import Property
import requests
import re
import selenium
from bs4 import BeautifulSoup, BeautifulStoneSoup
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# driver = webdriver.Chrome("C:\Users\Admin\Desktop\REAL_ESTATE_PREDICTION\SergeMartin_HousePrice_Prediction\data_acquisition\chromedriver.exe")
# driver.get ("https://www.immoweb.be/nl/zoeken/huis/te-koop?countries=BE")

# # #assigning url
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
url = "https://www.immoweb.be/nl/zoeken/huis/te-koop?countries=BE&priceType=SALE_PRICE&page=1&orderBy=relevance&gclid=Cj0KCQjw-fmZBhDtARIsAH6H8qjTKK4fMtFTGdZ8MLuXMXsZkgY9qfCZGz8AqwAO-e8qYmoK8WgHEqQaAhFgEALw_wcB"

#defining the get_property function to get  the property from the url
def get_property(url):
     req = requests.get(url, headers=headers)
     content=req.content
     soup=BeautifulSoup(content, "html.parser")
     scripts = soup.findAll('script', type='text/javascript')
     len(scripts)
     #print(len(scripts))
     property_dic = {} 
     for script in scripts:
        text = script.text
        if 'window.classified' in text:
            text = text[text.find('{'): text.rfind(';')]
            property_dic = json.loads(text)
            #print(property_dic)
            break
    # new_dic = {'id': property_dic ["id", "cluster"] }
    # print(new_dic)
     return property_dic["property"] # this dic contain info of all properties,
   
#Testing the above function on one web page
property_dic=get_property("https://www.immoweb.be/nl/zoekertje/huis/te-koop/berloz/4257/10151895?searchId=633d5556493a0")
#print(property_dic)

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.29M/6.29M [00:01<00:00, 6.06MB/s]
C:\Users\Admin\AppData\Local\Temp\ipykernel_21332\3871533204.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
from parsel import Selector 
#Obtain 10000 url of houses with webdriver
# driver = webdriver.Chrome("../chromedriver.exe")
# driver.get ("https://www.immoweb.be/nl/zoeken/huis/te-koop?countries=BE")
# The url  "houses_url" list.
# Iterate through all result pages (page) and get the url of each
houses_url = [] 
for page in range(1, 334):
    apikey = str(page)+'&orderBy=relevance'
    url = 'https://www.immoweb.be/en/search/house/for-sale?countries=BE&page='+apikey
    driver.implicitly_wait(10)
    driver.get(url)
    sel = Selector(text = driver.page_source) 
    # Store the xpath query of houses
    xpath_houses = '//*[@id="main-content"]/li//h2//a/@href'
    
    # Find nodes matching the xpath ``query`` and return the result
    page_houses_url = sel.xpath(xpath_houses).extract()
    
    # Add each page url list to houses_url, like in a matrix.
    houses_url.append(page_houses_url)

# Store all houses urls in a csv file
with open('C:/Users/Admin/Desktop/REAL_ESTATE_PREDICTION/SergeMartin_HousePrice_Prediction/data_acquisition/csv_files/houses_apartments_urls.csv', 'w') as file:
    for page_url in houses_url:
        for url in page_url:
            file.write(url+'\n')


# The url of each appartment that resulted from the search will be stored in the "houses_url" list
apartments_url = []
for page in range(1, 334):
    apikey = str(page)+'&orderBy=relevance'
    url = 'https://www.immoweb.be/en/search/apartment/for-sale?countries=BE&page='+apikey
    driver.implicitly_wait(10) 
    driver.get(url)
    sel = Selector(text=driver.page_source) 
    xpath_apartments = '//*[@id="main-content"]/li//h2//a/@href'
    page_apartments_url = sel.xpath(xpath_apartments).extract()
    apartments_url.append(page_apartments_url)
# As with houses, store all appartments urls in the same csv file
with open('C:/Users/Admin/Desktop/REAL_ESTATE_PREDICTION/SergeMartin_HousePrice_Prediction/data_acquisition/csv_files/houses_apartments_urls.csv', 'a') as file:
    for page_url in apartments_url:
        for url in page_url:
            file.write(url+'\n')
